# Wheat head detection
it is an essential focus of plant breeders for wheat variety selection and improved crop productivity.<a href="https://sourcerer.io/mmaithani"><img src="https://img.shields.io/badge/Python-124%20commits-orange.svg" alt=""></a>

![output](https://i.imgur.com/qgN9awQ.png)

# importing Modules

In [ ]:
import re
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
from PIL  import  Image
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv('../input/global-wheat-detection/train.csv')
sample_output = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')
train_dir = '../input/global-wheat-detection/train/'
test_dir ='../input/global-wheat-detection/test/'

In [ ]:
train.tail()

# Data exploration

- for now i dont know is there any insights are hidden from different sources

In [ ]:
print(train['source'].unique())

- width and height are of no use

In [ ]:
train.drop(['width','height'],axis=1, inplace= True)

- submission sample output :

In [ ]:
sample_output.head()

# How many [bounding-box](https://www.google.com/search?tbm=isch&q=machine%20learning%20bounding%20box) can one image contain ?
* image id = 35b935b6c	contains 116 bounding box
* on average one image contain 20-60 wheat heads

In [ ]:
train1=train
train1.drop(['source'],axis=1,inplace=True)
df = train1.drop_duplicates().groupby('image_id').count()
df.sort_values(by=['bbox'],ascending= False)


# Average wheat heads in a image : 20-60

In [ ]:
sns.distplot(df['bbox'])

# Some of the images are not annotated

In [ ]:
path, dirs, files = next(os.walk("../input/global-wheat-detection/train"))
file_count = len(files)
print("Total No. of images in  train directory :", file_count)
print('Total number of annotated images in train data :',len(train['image_id'].unique()),'\n')
print('--- > 49 images are unlabelled ')

## Extracting names of Unlabeled images

In [ ]:
# train directory images name
sep = '.'
images_dir=[]
for image in files:
    rest = image.split(sep, 1)[0]
    images_dir.append(rest)
    
# train data labeled images name
image_labelled =[]
for i in train['image_id']:
    image_labelled.append(i)

# extracting unlabeled images by comparing train dir. images and train.csv dataset      
unlabelled_images= [x for x in images_dir if x not in image_labelled]

print(len(unlabelled_images),'unlabelled images are :')
unlabelled=[]
for h in unlabelled_images:
    unlabelled.append(h+'.jpg')
# print(unlabelled)


# Can we Label the those images ?
## Let's see

In [ ]:
rows=7
cols=7
lis =[]
fig, axs = plt.subplots(rows, cols, figsize=(30,30))
for row in range(rows):
    for col in range(cols): 
        imgss= random.choice(unlabelled)
        img = Image.open(train_dir +imgss)
        axs[row, col].imshow(img)
        axs[row, col].axis('off')

>  no wheat heads in images :(  just grass

# Functions for plotting multiple bounding-box images

In [ ]:
# single box plotting
def plot():
    color = (0,0,255) 
    thickness = 2
    xmax =xmin+width
    ymax =ymin+height
    
    img = cv2.imread(train_dir+image_id+'.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img= cv2.rectangle(img,(xmin,ymin), (xmax,ymax),color,thickness)

    plt.figure(figsize=(20, 20))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    
def get_bbox(image_id):
    df=train.where(train['image_id']== image_id)
    df =df.dropna(axis='rows')
    arr = df["bbox"].to_numpy()
    return(arr)

def show_box(image_id):
    df=train.where(train['image_id']== image_id)
    df =df.dropna(axis='rows')
    arr = df["bbox"].to_numpy()

    img = cv2.imread(train_dir+image_id+'.jpg')
    # opencv always read image as BGR - conversion is must
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for box in arr:
        box=box[1:-1]
        xmin,ymin,width,height= box.split(",")
        # converting string to int
        xmin= int(float(xmin)); ymin= int(float(ymin)) 
        width= int(float(width)) ; height= int(float(height))

        xmax =xmin+width
        ymax =ymin+height

        img= cv2.rectangle(img,(xmin,ymin), (xmax,ymax),(255,0,0),2)
        
        img1 = Image.fromarray(img) # cv2 to PIL
    return img1
#     plt.figure(figsize=(20, 20))
#     plt.subplot(1, 2, 1)
#     plt.imshow(img1)

In [ ]:
show_box('134d4a01c')

# let's plot most and least bounding box images

In [ ]:
listt = ['ce4124be0','134d4a01c','a5cb30c38','35b935b6c','f1794c924','b70c20580','f79336a8e','22f341965','86296fc32']
rows=3
cols=3
i=0
fig, axs = plt.subplots(rows, cols, figsize=(20,20))
for row in range(rows):
    for col in range(cols): 
        img1=show_box(listt[i])
        axs[row, col].imshow(img1)    
        axs[row, col].axis('off')
        i=i+1

## Images containing Maximum wheat heads
> Note: some bbox are totally incorrect in future for better accuracy data needs to be cleaned

In [ ]:
listt=['35b935b6c','f1794c924','f79336a8e','134d4a01c','b70c20580','4cf91995d','026b6f389','c0e168cd3','690c7d8f0','a119a31fb',
       'a06241eb3','46e480374','b79a35c8d','bf2c0adde','3014e5cad','ab026459d','2cb361be8','c742fa9ae']
rows=3
cols=6
i=0
fig, axs = plt.subplots(rows, cols, figsize=(35,18))
for row in range(rows):
    for col in range(cols): 
        img1=show_box(listt[i])
        axs[row, col].imshow(img1)    
#         axs[row, col].axis('on')
        i=i+1